#### Initialize ####

In [5]:
function tf {aws-vault exec terraform_user -- terraform $args}

function prompt_to_confirm { 
    $user_response = Read-host "Continue? y/n or 1/0"
    if ($user_response -ne "y" -and $user_response -ne 1) {break}
}

In [4]:
$aws_region = "us-east-2"
$env:AWS_REGION=$aws_region #for aws_vault
$env:TF_VAR_aws_region=$aws_region #for terraform

In [2]:
(pwd).path

C:\Users\NOAksenov\OneDrive - DXC Production\Documents\_My\Code Repo\terraform\management_infrastructure\VPC1


In [3]:
terraform init #-upgrade

Initializing the backend...
Initializing modules...
- vpc in .terraform\modules\vpc
Initializing provider plugins...
- Finding hashicorp/aws versions matching ">= 5.46.0"...
- Installing hashicorp/aws v5.70.0...
- Installed hashicorp/aws v5.70.0 (signed by HashiCorp)
Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


#### Workflow ####

In [6]:
terraform fmt

main.tf


In [8]:
terraform validate

Success! The configuration is valid.



In [ ]:
tf plan -out=tfplan
terraform show -json tfplan > plan.json

In [ ]:
#Output planned resource addresses and their names in the cloud

$plan = Get-Content -Raw -Path "plan.json" | ConvertFrom-Json

$resourcesToDeploy = $plan.resource_changes | Where-Object { 
    $_.change.actions -contains "create" -or $_.change.actions -contains "update" 
} | ForEach-Object { 
    "Type: " + $_.type
    "TF name: " + $_.name
    "Cloud name: " + $_.change.after.name
    "Name tag: " + $_.change.after.tags_all.name
    ""
}

$resourcesToDeploy
Get-Date

In [9]:
###      ###
### PLAN ###
###      ###

tf plan
Get-Date

data.aws_availability_zones.available: Reading...
data.aws_availability_zones.available: Read complete after 1s [id=us-east-2]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # module.vpc.aws_default_network_acl.this[0] will be created
  + resource "aws_default_network_acl" "this" {
      + arn                    = (known after apply)
      + default_network_acl_id = (known after apply)
      + id                     = (known after apply)
      + owner_id               = (known after apply)
      + tags                   = {
          + "Example" = "example_tag"
          + "Name"    = "ex-VPC1-default"
        }
      + tags_all               = {
          + "Example" = "example_tag"
          + "Name"    = "ex-VPC1-default"
        }
      + vpc_id                 = (known after apply)

      + egress {
          + action          = 

In [ ]:
###       ###
### APPLY ###
###       ###

try {prompt_to_confirm} catch {break}
tf apply -auto-approve
Get-Date

In [ ]:
###         ###
### DESTROY ###
###         ###

try {prompt_to_confirm} catch {break}
tf destroy -auto-approve
Get-Date

#### Info ####

In [ ]:
terraform version

In [ ]:
#Root module configuration
cat .\main.tf

In [ ]:
($resources = terraform state list) | Out-Host

In [ ]:
foreach ($resource in $resources) { terraform state show $resource }

In [ ]:
terraform output

#### Troubleshooting ####

In [ ]:
#$env:TF_LOG = "TRACE"
#$env:TF_LOG_PATH = "log.txt"
#$env:TF_LOG = ""

In [ ]:
tf apply -refresh-only -auto-approve